# Title
#### Subtitle

Words

In [1]:
from blazingsql import BlazingContext
bc = BlazingContext()

BlazingContext ready


## MinIO

#### **Set Up MinIO Server**
MinIO can be deployed on Linux, Kubernetes, MacOS, Windows and with Source. This demo was built in an Ubuntu 18.04 environment, so we’ll set up MinIO for Linux.

Start by downloading the MinIO Server. The code cell below will download MinIO Server one level above where you are running this Notebook.

In [2]:
# wget -P .. https://dl.min.io/server/minio/release/linux-amd64/minio

Once the download completes, simply grant permissions with; 

In [3]:
# !chmod +x ../minio

#### **Launch MinIO Server**
MinIO Server installed, you’re now equipped to start a Server by calling `./minio server` followed the relative path to where you’d like the Server to run. In this case, we’re going to set up inside the `Welcome_to_BlazingSQL_Notebooks` repo. 

**Note**: running a continuous server in a Jupyter Notebook Code cell means that cell will execute until the server is turned off, so you will not be able to run the MinIO Server within this Notebook and complete the demo simultaneously. Please start a Terminal session with the JupyterLab Launcher and run the Server from there with `./minio server .`.

In [4]:
# !../minio server .

<img src="../data/imgs/minio_server.png" width="50%" />

#### **Register MinIO S3 bucker**
With the Server running in `Welcome_to_BlazingSQL_Noteooks`, all of the repo's sub-directories have automatically are accessable as S3 buckets.

For this demo, we'll use data stored in the `data` bucket. As this bucket is not public, we'll need to input our `access_key_id` and `secret_key`.

To help BlazingSQL locate and understand the MinIO bucket, we'll also input the Server's URL as the `endpoint_override` and the bucket's `encryption_type`.

`endpoint_override` is a new parameter that was added with [BlazingDB/blazingsql#524](https://github.com/BlazingDB/blazingsql/pull/524) to support MinIO S3 by extending our AWS S3 Storage Plugin to use custom URL endpoints.

In [5]:
from blazingsql import S3EncryptionType 

bc.s3('taxi', 
      bucket_name='data',
      access_key_id='minioadmin', 
      secret_key='minioadmin',
      encryption_type=S3EncryptionType.NONE, 
      endpoint_override="http://127.0.0.1:9000")

(True,
 '',
 OrderedDict([('type', 's3'),
              ('bucket_name', 'data'),
              ('access_key_id', 'minioadmin'),
              ('secret_key', 'minioadmin'),
              ('session_token', ''),
              ('encryption_type', <S3EncryptionType.NONE: 1>),
              ('kms_key_amazon_resource_name', ''),
              ('endpoint_override', 'http://127.0.0.1:9000')]))

#### **Create & Query table from MinIO S3 bucket**

Now that the MinIO S3 bucket is registered with BlazingContext, we can easily create & query tables from data stored there.

In [6]:
bc.create_table('taxi', 's3://taxi/sample_taxi.csv')

In [7]:
bc.sql("select * from taxi")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_x,pickup_y,dropoff_x,dropoff_y
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,1,N,1,12.0,1.0,0.5,3.25,0.00,0.3,17.05,-8236962.878,66.693057,-8234835.381,66.694056
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,1,N,1,14.5,0.5,0.5,2.00,0.00,0.3,17.80,-8237825.768,66.642117,-8237020.631,66.710783
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,1,N,2,9.5,0.5,0.5,0.00,0.00,0.3,10.80,-8233561.431,66.796854,-8232278.987,66.839490
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,1,N,2,3.5,0.5,0.5,0.00,0.00,0.3,4.80,-8238653.836,66.621592,-8238123.872,66.633735
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,1,N,2,15.0,0.5,0.5,0.00,0.00,0.3,16.30,-8234433.662,66.717321,-8238107.735,66.678369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743655,1,2015-01-28 11:11:43,2015-01-28 11:29:03,1,2.60,1,N,1,12.5,0.0,0.5,1.50,0.00,0.3,14.80,-8233738.934,66.725941,-8236360.724,66.662901
743656,1,2015-01-28 11:11:43,2015-01-28 11:24:39,1,2.50,1,N,2,12.0,0.0,0.5,0.00,0.00,0.3,12.80,-8236326.753,66.619617,-8239263.633,66.604966
743657,1,2015-01-28 11:11:43,2015-01-28 11:19:05,1,0.50,1,N,1,6.0,0.0,0.5,1.70,0.00,0.3,8.50,-8237036.767,66.674845,-8236477.079,66.687925
743658,1,2015-01-28 11:11:43,2015-01-28 11:35:15,1,9.10,1,N,1,28.5,0.0,0.5,6.92,5.33,0.3,41.55,-8223703.602,66.740356,-8233243.793,66.732021
